# Here we will download transcription factor binding sites and overlap them with promoters of genes

Take a look at the bedtools documentation


We need a file of transcription factor binding sites. We will get one from UCSC:

Go to the Table [Browser](https://genome.ucsc.edu/cgi-bin/hgTables)
1. Choose "clade: Mammal," "genome: Human," and "assembly: Feb. 2009 (GRCh37/hg19)"
2. Choose "group: Regulation", "track: Txn Fac ChIP V2"
3. Use the default table (there are no others anyway)
4. Put "chr22" for the Region
5. Choose "output format: BED - browser extensible data".
6. Save as "tf.bed"
7. Click "get output".
8. Do one bed record per gene (They really mean per "item", not a whole gene)
9. Click "get BED"
10. Use scp to move this file to TSCC. Think about where you want to put it. Maybe make a new folder in your projects directory called nfkb_binding

[This](https://genome.ucsc.edu/FAQ/FAQformat.html) is a good reference point for standar file formats. Check out  bed format.

In [15]:
%%bash

cd ~/test/

head ~/test/tf_binding.bed

chr22	16166497	16166741	CTCF	186	.	16166497	16166741	0	1	244,	0,
chr22	16201947	16202317	CTCF	603	.	16201947	16202317	0	1	370,	0,
chr22	16201988	16202252	YY1_(C-20)	63	.	16201988	16202252	0	1	264,	0,
chr22	16202021	16202231	Rad21	92	.	16202021	16202231	0	1	210,	0,
chr22	16202128	16202242	E2F6_(H-50)	269	.	16202128	16202242	0	1	114,	0,
chr22	16205233	16205683	SETDB1	356	.	16205233	16205683	0	1	450,	0,
chr22	16325696	16325900	NRSF	36	.	16325696	16325900	0	1	204,	0,
chr22	16560670	16560910	MafK_(ab50322)	102	.	16560670	16560910	0	1	240,	0,
chr22	16872258	16872628	CTCF	279	.	16872258	16872628	0	1	370,	0,
chr22	16872319	16872563	SMC3_(ab9263)	309	.	16872319	16872563	0	1	244,	0,


extract the rows that contain data on nkfb

In [7]:
%%bash 

grep 'NFKB' ~/test/tf_binding.bed > nfkb_binding.bed

#count the number of lines in the file with wc -l
wc -l nfkb_binding.bed

702 nfkb_binding.bed


make an annotation file that only contains chromosome 22 - this is for space and time purposes

In [12]:
%%bash

grep 'chr22' gencode.v19.annotation.gtf > gencode.v19.annotation.chr22.gtf

head /home/ecwheele/scratch/test_annotations/gencode.v19.annotation.chr22.gtf 

chr22	HAVANA	gene	16062157	16063236	.	+	.	gene_id "ENSG00000233866.1"; transcript_id "ENSG00000233866.1"; gene_type "lincRNA"; gene_status "KNOWN"; gene_name "LA16c-4G1.3"; transcript_type "lincRNA"; transcript_status "KNOWN"; transcript_name "LA16c-4G1.3"; level 2; havana_gene "OTTHUMG00000140195.1";
chr22	HAVANA	transcript	16062157	16063236	.	+	.	gene_id "ENSG00000233866.1"; transcript_id "ENST00000424770.1"; gene_type "lincRNA"; gene_status "KNOWN"; gene_name "LA16c-4G1.3"; transcript_type "lincRNA"; transcript_status "KNOWN"; transcript_name "LA16c-4G1.3-001"; level 2; tag "basic"; havana_gene "OTTHUMG00000140195.1"; havana_transcript "OTTHUMT00000276574.1";
chr22	HAVANA	exon	16062157	16062316	.	+	.	gene_id "ENSG00000233866.1"; transcript_id "ENST00000424770.1"; gene_type "lincRNA"; gene_status "KNOWN"; gene_name "LA16c-4G1.3"; transcript_type "lincRNA"; transcript_status "KNOWN"; transcript_name "LA16c-4G1.3-001"; exon_number 1; exon_id "ENSE00001660730.1"; level 2; tag "basic"; h

**Introduction to awk**

Another way to extract all lines

    awk -F "\t" '{print;}' knownGene.txt
What if we only wanted one column

    awk -F "\t" '{print $8;}' knownGene.txt  | head
What if we wanted the length of genes?

    awk -F "\t" '{ len = $5-$4;} {print len;}' knownGene.txt | head
Length of all genes summed?

    awk -F "\t" '{ len = $5-$4;} {tot = tot + len;} END {print tot;}' knownGene.txt | head
Don't process the header line (introduction to conditionals)

    awk -F "\t" '{
    if (FNR==1){
        next
    };
    tot = tot + $5-$4};
    END {print tot;}' knownGene.txt | head
    What if you only want the total length of genes in chromosome 1?

    awk -F "\t" '{
        if (FNR==1){
            next;
            };
        chr =$2;
        if (chr == "chr1") {
            tot = tot + $5-$4;
        }
    };
    END {print tot;}' knownGene.txt

In [24]:
%%bash

awk -F "\t" '{if($3 == "transcript") {print $0}}' ~/scratch/test_annotations/gencode.v19.annotation.chr22.gtf > gencode.v19.annotation.chr22.transcript.gtf


**Use flank to get promoter regions for genes** 

Get 2000bp upstream from each gene. The genome we are using is "hg19.genome", provided in the mtsp_bootcamp directory. Save this file as gencode.v19.annotation.chr22.transcript.promoter.gtf

Go to the documentation for bedtools and find the entry for "bedtools flank". bedtools flank -h is also informative (below)

$ bedtools flank -h

Tool:    bedtools flank (aka flankBed)
Version: v2.22.1
Summary: Creates flanking interval(s) for each BED/GFF/VCF feature.

Usage:   bedtools flank [OPTIONS] -i <bed/gff/vcf> -g <genome> [-b <int> or (-l and -r)]

Options: 
        -b      Create flanking interval(s) using -b base pairs in each direction.
                - (Integer) or (Float, e.g. 0.1) if used with -pct.

        -l      The number of base pairs that a flank should start from
                orig. start coordinate.
                - (Integer) or (Float, e.g. 0.1) if used with -pct.

        -r      The number of base pairs that a flank should end from
                orig. end coordinate.
                - (Integer) or (Float, e.g. 0.1) if used with -pct.

        -s      Define -l and -r based on strand.
                E.g. if used, -l 500 for a negative-stranded feature, 
                it will start the flank 500 bp downstream.  Default = false.

        -pct    Define -l and -r as a fraction of the feature's length.
                E.g. if used on a 1000bp feature, -l 0.50, 
                will add 500 bp "upstream".  Default = false.

        -header Print the header from the input file prior to results.

Notes: 
        (1)  Starts will be set to 0 if options would force it below 0.
        (2)  Ends will be set to the chromosome length if requested flank would
        force it above the max chrom length.
        (3)  In contrast to slop, which _extends_ intervals, bedtools flank
        creates new intervals from the regions just up- and down-stream
        of your existing intervals.
        (4)  The genome file should tab delimited and structured as follows:

        <chromName><TAB><chromSize>

        For example, Human (hg19):
        chr1    249250621
        chr2    243199373
        ...
        chr18_gl000207_random   4262

Tips: 
        One can use the UCSC Genome Browser's MySQL database to extract
        chromosome sizes. For example, H. sapiens:

        mysql --user=genome --host=genome-mysql.cse.ucsc.edu -A -e \
        "select chrom, size from hg19.chromInfo"  > hg19.genome

**Use bedtools intersect to overlap promoters with nfkb binding sites**

Take a look at the diagrams in the documentation for bedtools intersect and think about how it can answer these questions:

Given locations of genome methylation, which genes does it overlap?

Given locations of RBP binding, which exons does it overlap?

Given two CHIP-Seq experiments, which peaks are consistent between them?

Use bedtools intersect to find which promoters overlap with the NFKB binding sites. Use the promoters as "A" and the binding sites as "B". Call this file gencode.v19.annotation.chr22.transcript.promoter.nfkb.gtf.

**Use bedtools getfasta to extract sequences**
Read the documentation for bedtools getfasta and figure out how to request the sequences in fasta format for the peaks which overlap in gene promoters. Does the strand matter?